In [ ]:
import sys
sys.path.append('../scripts')
from scripts.sentiment_anaysis import get_sentiment_score
from scripts.script_preprocess import preprocess
from scripts.text_classfication import classifier
from scripts.specfic_bank_topkeyword import specfic_bank_keyword

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import spacy

2025-06-20 13:45:50.356827: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750427150.404586     531 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750427150.418575     531 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
all_review_df = pd.read_csv("src/data/raw_review.csv")

In [19]:
all_review_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1200 entries, 0 to 399
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   source_id  1200 non-null   object        
 1   review     1200 non-null   object        
 2   rating     1200 non-null   int64         
 3   date       1200 non-null   datetime64[ns]
 4   bank       1200 non-null   object        
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 56.2+ KB


In [20]:
all_review_df.to_csv("All_bank_review.csv" , index=False)

* Do sentment analysis

In [22]:
all_review_df["sentiment_score"] = all_review_df["review"].apply(get_sentiment_score)

In [23]:
all_review_df["sentiment_label"] = all_review_df["sentiment_score"].apply(lambda x : 'postive' if x > 0 else( "neutral" if x==0  else 'negative') )

In [24]:
all_review_df.head()

,source_id,review,rating,date,bank,sentiment_score,sentiment_label
0,78b9faa5-11d1-48a5-aede-3fc0b7ffd97e,SUFaa milikesaa,5,2025-06-19,CBE,0.000000,neutral
1,524266cc-a927-4908-bd3d-c35ea46ca389,"Am not able to make screenshot,",1,2025-06-18,CBE,-0.250000,negative
2,59c4f060-eda8-4a7a-8712-126978051c97,best app and bank to use,5,2025-06-18,CBE,1.000000,postive
3,d43ab2b9-3b6c-473b-a8db-59f171fab8b7,"Verily, it was very superior competitive to we...",5,2025-06-18,CBE,0.348889,postive
4,a9aca89a-7a9d-4f52-9ecb-ce75a15adbb1,thank you,5,2025-06-17,CBE,0.000000,neutral


* Script preprocessing 

In [25]:
nlp = spacy.load("en_core_web_sm")

* Extract keywords and Get top keywords

In [29]:
CBE_top_keyword = specfic_bank_keyword(all_review_df , "CBE"  )
CBE_top_keyword.visualize_barchart(top_n_topics=5).show()

In [30]:
BOA_top_keyword = specfic_bank_keyword(all_review_df , "BOA"  )
BOA_top_keyword.visualize_barchart(top_n_topics=5).show()

In [31]:
Dashen_top_keyword = specfic_bank_keyword(all_review_df , "Dashen"  )
Dashen_top_keyword.visualize_barchart(top_n_topics=5).show()

In [33]:
themes = [
    "Account Access Issues",
    "Transaction Performance",
    "User Interface & Experience",
    "Customer Support",
    "Feature Requests"
]


def get_theme(review):
    result = classifier(review, themes)
    return result['labels'][0], result['scores'][0] 
    
theme_results = all_review_df["review"].apply(get_theme)

all_review_df["identified_theme"] = theme_results.apply(lambda x: x[0])
all_review_df["theme_score"] = theme_results.apply(lambda x: x[1])


In [34]:
all_review_df.head()

,source_id,review,rating,date,bank,sentiment_score,sentiment_label,cleaned_text,identified_theme,theme_score
0,78b9faa5-11d1-48a5-aede-3fc0b7ffd97e,SUFaa milikesaa,5,2025-06-19,CBE,0.000000,neutral,appsuddenlystop,Transaction Performance,0.322818
1,524266cc-a927-4908-bd3d-c35ea46ca389,"Am not able to make screenshot,",1,2025-06-18,CBE,-0.250000,negative,niceapp,User Interface & Experience,0.297540
2,59c4f060-eda8-4a7a-8712-126978051c97,best app and bank to use,5,2025-06-18,CBE,1.000000,postive,supersuperapp,User Interface & Experience,0.358189
3,d43ab2b9-3b6c-473b-a8db-59f171fab8b7,"Verily, it was very superior competitive to we...",5,2025-06-18,CBE,0.348889,postive,helpful/ctransfersaveusablefuturetimetransfer,Customer Support,0.352848
4,a9aca89a-7a9d-4f52-9ecb-ce75a15adbb1,thank you,5,2025-06-17,CBE,0.000000,neutral,trulystepahead,User Interface & Experience,0.335298


In [35]:
all_review_df.to_csv("Analysied_review.csv" , index=False)